In [1]:
import nltk #import library nltk
from nltk.tokenize import word_tokenize #import word_tokenize for tokenizing text into words 
from nltk.tokenize import sent_tokenize #import sent_tokenize for tokenizing paragraph into sentences
from nltk.corpus import stopwords #import stopwords
from nltk.tokenize import RegexpTokenizer

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
import time

#library for regular expretion
import re

In [2]:
def stpword(str):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(str) 

    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    filtered_sentence = [] 

    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w) 
    return ' '.join(filtered_sentence)

In [3]:
data = pd.read_excel("dataset.xlsx")
data.columns = ['NEWS', 'TYPE']

In [4]:
for i in range(len(data)):
    #remove punc, num, sym
    data.NEWS[i] = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))', '', data.NEWS[i])
    data.NEWS[i] = re.sub(r'[\W,0-9,(_),($),(%),('')]', ' ', data.NEWS[i])
    data.NEWS[i] = re.sub(' +',' ',data.NEWS[i])
    data.NEWS[i] = re.sub('[^A-Za-z]', ' ', data.NEWS[i])
    data.NEWS[i] = data.NEWS[i].replace("-"," ")
    data.NEWS[i] = data.NEWS[i].replace("\n"," ")
    stop = re.compile(r'\W*\b\w{1}\b')
    
    data.NEWS[i] = stop.sub('',data.NEWS[i])
    
    data.NEWS[i] = data.NEWS[i].lower()
    
    data.NEWS[i] = stpword(data.NEWS[i])

In [5]:
data['ID_TYPE'] = data.TYPE.map({'business':0, 'entertainment':1, 'politics':2, 'sport':3, 'tech':4})

In [6]:
x_train, x_test, y_train, y_test = train_test_split(data.NEWS, data.ID_TYPE, test_size=0.4, random_state=0)

In [7]:
cv = CountVectorizer()
X_train = cv.fit_transform(x_train)
X_test = cv.transform(x_test)

In [8]:
multiNB = MultinomialNB()
multiNB.fit(X_train, y_train)
result = multiNB.predict(X_test)

In [9]:
accuracy_score(result, y_test)

0.9719101123595506

In [10]:
def predict_news(dt):
    test = cv.transform(dt)
    pred= multiNB.predict(test)
    if pred  == 0:
         return 'Business'
    elif pred == 1:
        return 'Entertainment'
    elif pred == 2:
        return 'Politics'
    elif pred == 3:
        return 'Sport'
    else:
        return 'Tech'

In [11]:
x = ["Margaret Atwood to write Handmaid's Tale sequel inspired by modern America"]
r = predict_news(x)
print (r)

Entertainment
